In [ ]:
# Test verilerini manuel accuracy hesaplama için hazırlama

import tensorflow
from keras.layers import Reshape
from keras.layers import Input, Embedding, Reshape, LSTM, Dense, Flatten, Bidirectional, Concatenate
from keras.models import Model
import os
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import re
import pickle


model_path='/content/drive/MyDrive/Colab Notebooks/bi_best_model.keras' # modelin konumu
model_test_path='/content/drive/MyDrive/Colab Notebooks/accuracy_calculation/unlabeled_test_data'# modelin testi için kullanılacak etiketsiz test verilerinin konumu
model_test_path_labeled='/content/drive/MyDrive/Colab Notebooks/accuracy_calculation/labeled_test_data' # model testinin accuracy hesaplanması için kullanılacak etiketli verilerin konumu



global words,wordsIndext

words = []
wordsIndext = []
max_sequence_length = 384



label_list = ["<start_\"company_name\">",
              "<end_\"company_name\">",
                  "<start_\"date\">",
                  "<end_\"date\">",
                  "<start_\"time\">",
                  "<end_\"time\">",
                  "<start_\"receipt_number\">",
                  "<end_\"receipt_number\">",
                  "<start_\"tax\">",
                  "<end_\"tax\">",
                  "<start_\"amount\">",
                  "<end_\"amount\">"]

label_list2= ["Pad","Others","B_Comp","l_Comp","B_Date","l_Date","B_Time","l_Time",
                  "B_Receipt","l_Receipt","B_Tax","l_Tax","B_Amount","l_Amount"]


#gonna be added as code
global arrayOfAccuracyValues
arrayOfAccuracyValues = [] #["correctTotal",Total]

for i,element in enumerate(label_list2):
    if(i>0):
        arrayOfAccuracyValues+=[[0,0,0,0,0]]
print (arrayOfAccuracyValues)


def splitfunction(text:str):
    global words,wordsIndext
    words = []
    wordsIndext = []
    texts = text.split()
    for word in texts:
        words.append(word)
        wordsIndext.append(1)
    for i in range(max_sequence_length-len(wordsIndext)):
        wordsIndext.append(0)



def indexAssignment(tag,last):
    if(tag==None):
        return None
    index = label_list.index(tag)+2
    if(last == "l"):
       index +=1
    return index


def checkTag(word:str):
    for tag in label_list:
        index = word.find(tag)
        if index != -1:
            return tag
    return None



def tagingWords():
    control = None
    counter = 0
    for word in words:
        tag = checkTag(word)
        if tag != None:
            if(word.find("<start_") != -1):
                control = tag
            if(word.find("<start_") != -1 and word.find("<end_") == -1):
                wordsIndext[counter] = indexAssignment(tag,"B")
                counter+=1
            elif(word.find("<start_") != -1 and word.find("<end_") != -1):
                wordsIndext[counter] = indexAssignment(tag,"B")
                counter+=1
            else:
                wordsIndext[counter] = indexAssignment(tag,"B")
                counter+=1
            if(word.find("<end_") != -1):
                control = None
        elif control !=None:
            wordsIndext[counter] = indexAssignment(control,"l")
            counter+=1
        else:
            counter+=1


def indexListFunction(texts):
    splitfunction(texts)
    tagingWords()
    return wordsIndext


folder_path_test =  model_test_path_labeled
file_list_test = os.listdir(folder_path_test)
accuracy_testing_data = []

for file_name in file_list_test:
    if file_name.endswith(".txt"):
        file_path = os.path.join(folder_path_test, file_name)
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
            result = indexListFunction(text)
            if len(accuracy_testing_data) == 0:
                accuracy_testing_data = np.array([result])
            else:
                accuracy_testing_data = np.concatenate((accuracy_testing_data, [result]), axis=0)



#gonna be added as code
def check_all_data_inreceipt_spesific_label(npArray,npArrayLabeled,labelIndex):
    global arrayOfAccuracyValues
    for i,element in enumerate(npArray):
        if element == labelIndex:
            arrayOfAccuracyValues[labelIndex-1][1]+=1
            if(npArrayLabeled[i]==labelIndex):
                arrayOfAccuracyValues[labelIndex-1][0]+=1     # doğru etiketlenenler
            elif(npArrayLabeled[i]!=labelIndex and npArrayLabeled[i]==1):
                arrayOfAccuracyValues[labelIndex-1][2]+=1     # yanlış etiketlenmiş ve other olanlar
            elif(npArrayLabeled[i]!=labelIndex):
                arrayOfAccuracyValues[labelIndex-1][3]+=1     # yanlış etiketlenmiş ve other olmayan



global dataAccuracyArray
dataAccuracyArray = [0 for _ in range(13)]

def compute_data_accuracy_ratio():
    for i,element in enumerate(arrayOfAccuracyValues):
        if(element[1]!=0):
            dataAccuracyArray[i] = round(element[0]/element[1],4)
        else:
            dataAccuracyArray[i] = 0.0


def accuracy_calculation(etiketli_gercek_fis,model_tahmini_fis):
    for i,element in enumerate(label_list2[1:]):
        check_all_data_inreceipt_spesific_label(etiketli_gercek_fis, model_tahmini_fis, i+1)



#MODELİ ÇALIŞTIR
def model_run(directory):
    all_texts = []

    for filename in os.listdir(directory):
        if filename.endswith('.txt'):
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
                txt_data = content.split()
                all_texts.append(txt_data)


    #TOKENIZER YÜKLE
    with open('/content/drive/MyDrive/Colab Notebooks/tokenizer.pickle', 'rb') as handle:
        tokenizer_model = pickle.load(handle)


    all_sequences=[]
    for i in range(len(all_texts)):
        all_sequences.append(tokenizer_model.texts_to_sequences(all_texts[i]))

    max_word_length,trunc_type,padding_type = 20,'post','post'

    all_padded_sequences=[]
    for i in range(len(all_sequences)):
        padded_sequence = pad_sequences(all_sequences[i], maxlen=max_word_length, padding=padding_type, truncating=trunc_type)
        all_padded_sequences.append(padded_sequence)

    padded_arrays = []
    for arr in all_padded_sequences:
        pad_width = ((0, max_sequence_length - len(arr)), (0, 0))
        padded_arr = np.pad(arr, pad_width, mode='constant', constant_values=0)
        padded_arrays.append(padded_arr)

    xtrain=np.array(padded_arrays)
    model = tensorflow.keras.models.load_model(model_path)

    prediction=model.predict(xtrain)
    #------------------------------------------------------------
    label_list = ["Pad", "Others", "B_Comp", "I_Comp", "B_Date", "I_Date", "B_Time", "I_Time", "B_Receipt", "I_Receipt", "B_Tax", "I_Tax", "B_Amount", "I_Amount"]

    predicted_labels = np.argmax(prediction, axis=-1)


    # gonna be added
    for i,element in enumerate(predicted_labels):
        model_tahmini_fis=predicted_labels[i]
        etiketli_gercek_fis = accuracy_testing_data[i]
        accuracy_calculation(etiketli_gercek_fis,model_tahmini_fis)
    compute_data_accuracy_ratio()

    for i,element in enumerate(label_list):   # th değeri hesaplaması
        if(i == 0):
            continue
        arrayOfAccuracyValues[i-1][4] = arrayOfAccuracyValues[i-1][1] - arrayOfAccuracyValues[i-1][0]

    print("-------------------------------------------------------------------------------")

    for i,element in enumerate(label_list):
        if(i == 0):
            continue
        print(f"{element} : {dataAccuracyArray[i-1]}") 

    print("-------------------------------------------------------------------------------")

    print("[Doğru bilinenler,   Tüm tahminler,     yanlış olup others olanlar,   yanlış olup others olmayanlar,     yanlış tahminler]")


    for i,element in enumerate(label_list):
        if(i == 0):
            continue
        print(f"{element} : {arrayOfAccuracyValues[i-1]}")  # gerçek veri ile model çıktısının karşılaştırıp accuracy döndürür


model_run(model_test_path)
